In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sb
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
data_new = pd.read_csv('/kaggle/input/santander-pr/train.csv', dtype={"ind_ahor_fin_ult1":"int8",'ind_aval_fin_ult1':"int8", 'ind_cco_fin_ult1':"int8", 
                                                                  'ind_cder_fin_ult1':"int8", 'ind_cno_fin_ult1':"int8", 'ind_ctju_fin_ult1':"int8", 'ind_ctma_fin_ult1':"int8", 
                                                                  'ind_ctop_fin_ult1':"int8", 'ind_ctpp_fin_ult1':"int8",
                                                                  'ind_nomina_ult1':"float16", 'ind_nom_pens_ult1':"float16",
                                                                  'ind_actividad_cliente':"float16", 'cod_prov':"float16",
                                                                  'tipodom':"float16", "indrel":"float16", "ind_nuevo":"float16",
                                                                  'ind_deco_fin_ult1':"int8", 'ind_deme_fin_ult1':"int8", 'ind_dela_fin_ult1':"int8",
                                                                  'ind_ecue_fin_ult1':"int8", 'ind_fond_fin_ult1':"int8", 'ind_hip_fin_ult1':"int8",
                                                                  'ind_plan_fin_ult1':"int8", 'ind_pres_fin_ult1':"int8", 'ind_reca_fin_ult1':"int8", 'ind_tjcr_fin_ult1':"int8", 
                                                                  'ind_valo_fin_ult1':"int8", 'ind_viv_fin_ult1':"int8", 'ind_recibo_ult1':"int8"},
                      parse_dates=['fecha_dato', 'fecha_alta', 'ult_fec_cli_1t'])

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5,8,11,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data_test = pd.read_csv('/kaggle/input/santander-pr/test.csv')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (11,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
data_test.shape

(931453, 24)

In [5]:
pd.set_option('display.max_columns', 100)

In [6]:
unique_ids   = pd.Series(data_new["ncodpers"].unique())
limit_people = 951952
unique_id    = unique_ids.sample(n=limit_people, replace=True)
data_new           = data_new[data_new.ncodpers.isin(unique_id)]

In [58]:
unique_ids.size

(951952,)

In [7]:
data_new['age'] = pd.to_numeric(data_new['age'], errors='coerce') # errors = 'coerce' will convert the rows having 'NA' to NaN

In [8]:
mean_age = float(data_new['age'].mean())
mean_age = round(mean_age)
mean_age

40

In [9]:
data_new['age'].fillna(value = mean_age, inplace = True)


In [10]:
data_new.dropna(subset=['fecha_alta'], inplace=True)

In [11]:
data_new.dropna(subset=['indrel_1mes'], inplace=True)


In [12]:
data_new.drop(columns = ['nomprov'], inplace = True)

In [13]:
data_new.loc[data_new["cod_prov"].isnull(), "cod_prov"] = 28.0


In [14]:
data_new.loc[data_new["canal_entrada"].isnull(), "canal_entrada"] = "KHE"

In [ ]:
conditions = [
    (data_new['age'] <= 10),
    (data_new['age'] > 10) & (data_new['age'] <= 20),
    (data_new['age'] > 20) & (data_new['age'] <= 26),
     (data_new['age'] > 26) & (data_new['age'] <= 50),
    (data_new['age'] > 50) & (data_new['age'] <= 90),
    (data_new['age'] > 90)
    ]
values = ['1', '2', '3', '4', '5', '6']


In [ ]:
data_new['AgeClass'] = np.select(conditions, values)

AgeClass
1    130418.40
2    103326.36
3     95962.74
4    120833.31
5    120057.99
Name: renta, dtype: float64

In [15]:
age_groups = data_new.groupby("AgeClass")
age_groups['renta'].median()

# Or apply the "max" function to the age column
#age_groups['renta'].max()


KeyError: 'AgeClass'

In [18]:

conditions  = [ 
    (data_new['age'] <= 10),
    (data_new['age'] > 10) & (data_new['age'] <= 20),
    (data_new['age'] > 20) & (data_new['age'] <= 26),
     (data_new['age'] > 26) & (data_new['age'] <= 50),
    (data_new['age'] > 50) & (data_new['age'] <= 90),
    (data_new['age'] > 90)
    ] 
choices     = [130371.75,  100867.32, 88513.47, 100966.50, 121533.63, 120668.85]
    
data_new["rentaAGeGroup"] = np.select(conditions, choices, default=np.nan)

100823.76


89232.87


In [19]:
data_new["renta"].fillna(data_new["rentaAGeGroup"], inplace=True)

In [20]:
data_new['ind_empleado'] = pd.to_numeric(data_new['ind_empleado'], errors='ignore')

In [21]:
condlist = [data_new['ind_empleado'] == 'N', data_new['ind_empleado'] == 'nan', data_new['ind_empleado'] == 'A',
            data_new['ind_empleado'] == 'B', data_new['ind_empleado'] == 'F', data_new['ind_empleado'] == 'S'] 
choicelist = ['N', "Unknown", 'A', 'B', 'F', 'S'] 

In [22]:
data_new['empleado'] = np.select(condlist, choicelist)

In [ ]:
# from category_encoders import *
# alcdata = BinaryEncoder().fit_transform(alcdata)

In [23]:
data_new.dropna(subset=['ind_nom_pens_ult1', 'ind_nom_pens_ult1'], inplace = True)

In [24]:
data_new.drop(columns=['ult_fec_cli_1t'], inplace= True)

In [25]:
data_new.drop(columns=['tipodom'], inplace= True)

In [26]:
data_new.loc[data_new["conyuemp"].isnull(), "conyuemp"] = 0


In [27]:
data_new.loc[data_new['sexo'].isnull(), 'sexo'] = 'V'

In [28]:
data_new.dropna(subset=['segmento'], inplace = True)

In [ ]:
data_test.shape

In [ ]:
data_new.columns

In [29]:
data_new.drop(columns=['indext', 'tiprel_1mes'], inplace=True)


In [30]:
lists= []
cols = data_new.columns
for col in cols:
    if(data_new[col].dtype == 'object'):
        lists.append(col)

In [ ]:
lists

In [ ]:
list_new1 = lists[:5]
list_new2 = lists[5:]
print(list_new1, list_new2)


In [ ]:
from category_encoders import *
# data_new = BinaryEncoder().fit_transform(data_new)
data_dummies = pd.get_dummies(data_new, columns = list_new1)



In [ ]:
data_dummies = pd.get_dummies(data_dummies, columns = list_new2)


In [31]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for feat in lists:
    data_new[feat] = le.fit_transform(data_new[feat].astype(str))

In [ ]:
data_new.isnull().sum()

In [ ]:
data_new.columns

In [32]:
listColsDrop = ['rentaAGeGroup', 'empleado']
data_new.drop(columns=listColsDrop, inplace= True)

In [33]:
y = data_new.iloc[:, 19:]
X = data_new.iloc[:,:19]

In [34]:
import datetime as dt
X['fecha_dato'] = pd.to_datetime(X['fecha_dato'])
X['fecha_dato']=X['fecha_dato'].map(dt.datetime.toordinal)
X['fecha_alta'] = pd.to_datetime(X['fecha_alta'])
X['fecha_alta']=X['fecha_alta'].map(dt.datetime.toordinal)

In [ ]:
print(X.shape)
print(y.shape)

In [35]:
del data_new

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [37]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()


In [ ]:
data_new.isnull().sum()

In [ ]:
X_train.isnull().sum()

In [38]:
clf = clf.fit(X_train, y_train)

In [39]:
y_predict = clf.predict(X_test)


In [40]:
y_test.shape

(3171090, 24)

In [58]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_predict)

0.9145820214043677

In [ ]:
y_predict_train = clf.predict(X_train)


In [ ]:
accuracy_score(y_train, y_predict_train)

In [ ]:
y_predict

In [41]:
targetcols = ['ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
       'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
       'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
       'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
       'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
       'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
       'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
       'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']

In [42]:
dtype_list = {'ind_cco_fin_ult1': 'float16',
              'ind_deme_fin_ult1': 'float16',
              'ind_aval_fin_ult1': 'float16',
              'ind_valo_fin_ult1': 'float16',
              'ind_reca_fin_ult1': 'float16',
              'ind_ctju_fin_ult1': 'float16',
              'ind_cder_fin_ult1': 'float16', 
              'ind_plan_fin_ult1': 'float16',
              'ind_fond_fin_ult1': 'float16', 
              'ind_hip_fin_ult1': 'float16',
              'ind_pres_fin_ult1': 'float16', 
              'ind_nomina_ult1': 'float16', 
              'ind_cno_fin_ult1': 'float16',
              'ncodpers': 'int64',
              'ind_ctpp_fin_ult1': 'float16',
              'ind_ahor_fin_ult1': 'float16',
              'ind_dela_fin_ult1': 'float16',
              'ind_ecue_fin_ult1': 'float16',
              'ind_nom_pens_ult1': 'float16',
              'ind_recibo_ult1': 'float16',
              'ind_deco_fin_ult1': 'float16',
              'ind_tjcr_fin_ult1': 'float16', 
              'ind_ctop_fin_ult1': 'float16',
              'ind_viv_fin_ult1': 'float16',
              'ind_ctma_fin_ult1': 'float16'}       



In [ ]:
len(final_preds)

In [ ]:
len(test_id)

In [ ]:
len(final_preds)

In [ ]:
y_test

In [ ]:
len(test_id)

In [ ]:
len(final_preds)

In [ ]:
targetcols[0]


In [ ]:
print("Getting the top products..")
target_cols = np.array(targetcols)
preds = np.argsort(y_predict, axis=1)
preds = np.fliplr(preds)[:,:8]
test_id = np.array(pd.read_csv('../input/santander-pr/test.csv', usecols=['ncodpers'])['ncodpers'])
final_preds = [" ".join(list(target_cols[pred])) for pred in preds]
out_df = pd.DataFrame({'ncodpers':test_id, 'added_products':final_preds})
out_df.to_csv('sub_xgb_new.csv', index=False)
print(datetime.datetime.now()-start_time)

In [ ]:
train_size = 13647309
nrows = 1000000
start_idx = train_size - nrows
start_idx

In [43]:
last_instance_df = pd.read_csv('../input/santander-pr/train.csv', usecols=['ncodpers']+targetcols, dtype=dtype_list)
last_instance_df = last_instance_df.drop_duplicates('ncodpers', keep='last')
last_instance = np.array(last_instance_df.fillna(0)).astype('int')

In [42]:
last_instance_df.shape

(951952, 25)

In [ ]:
targetcols.shape

In [ ]:
test_df.head()

In [44]:
last_instance_df.fillna(0,inplace=True)
recommendation_product = {}
targetcols = np.array(targetcols)
for idx,row_val in last_instance_df.iterrows():
    #print(last_instance_df)
    ids = row_val['ncodpers']
    #print(ids)
    used_product = set(targetcols[np.array(row_val[1:]) == 1])
    #print(used_product)
    recommendation_product[ids] = used_product
    #print(recommendation_product[ids])
    

In [46]:
print(len(used_product))

1


In [41]:
pred = np.argsort(y_predict,axis=1)

In [42]:
pred = np.fliplr(pred) 


In [ ]:
pred.shape

In [ ]:
test_ids.shape 931453

In [ ]:
X_train.shape

In [47]:
data_test1 = pd.read_csv('../input/santander-pr/test.csv')
data_test1.head()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (11,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento
0,2016-05-28,657640,N,ES,H,68,2006-11-17,0.0,114,1.0,NaN,1,I,S,N,NaN,KAQ,N,1.0,28.0,MADRID,0.0,147347.97,02 - PARTICULARES
1,2016-05-28,657788,N,ES,H,42,2006-11-18,0.0,114,1.0,NaN,1,A,S,N,NaN,KBF,N,1.0,28.0,MADRID,1.0,132559.35,02 - PARTICULARES
2,2016-05-28,657795,N,ES,V,44,2006-11-18,0.0,114,1.0,NaN,1,A,S,N,NaN,KAP,N,1.0,26.0,"RIOJA, LA",1.0,81399.57,02 - PARTICULARES
3,2016-05-28,657790,N,ES,V,42,2006-11-18,0.0,114,1.0,NaN,1,A,S,N,NaN,KBF,N,1.0,48.0,BIZKAIA,1.0,NaN,02 - PARTICULARES
4,2016-05-28,657794,N,ES,H,49,2006-11-18,0.0,114,1.0,NaN,1,A,S,N,NaN,KAT,N,1.0,8.0,BARCELONA,1.0,102189.00,01 - TOP


In [ ]:
X_train.isnull().sum()

In [48]:
data_test1.drop(columns= ['nomprov', 'ult_fec_cli_1t', 'tipodom', 'indext', 'tiprel_1mes'], inplace = True)

In [ ]:
data_test1.isnull().sum()

In [51]:
data_test1.loc[data_test1['sexo'].isnull(), 'sexo'] = 'V'
data_test1.loc[data_test1["conyuemp"].isnull(), "conyuemp"] = 0
data_test1.loc[data_test1["cod_prov"].isnull(), "cod_prov"] = 28.0
data_test1.loc[data_test1["canal_entrada"].isnull(), "canal_entrada"] = "KHE"
#data_test1.dropna(subset=['indrel_1mes'], inplace=True)
#data_test1.dropna(subset=['segmento'], inplace=True)
data_test1.loc[data_test1["renta"].isnull(), "renta"] = meanRenta



In [50]:
meanRenta = data_test1.renta.mean()
meanRenta

134089.6370984329

In [52]:
lists= []
cols = data_test1.columns
for col in cols:
    if(data_test1[col].dtype == 'object'):
        lists.append(col)

In [53]:
le = LabelEncoder()

for feat in lists:
    data_test1[feat] = le.fit_transform(data_test1[feat].astype(str))

In [ ]:
data_test1.shape

In [54]:
y_test1 = clf.predict(data_test1)

In [ ]:
y_test1.shape

In [ ]:
targetcols

In [ ]:
data_test1['ncodpers'].nunique()

In [ ]:
#for i in y_test1:
#    row = i
ncod = data_test1.loc[:,"ncodpers"]
#print(ncod)
for idx,predicted in enumerate(y_test1):
    print(idx, ncod[idx])
    
    break

    

In [56]:
pred = np.argsort(y_test1,axis=1)
pred = np.fliplr(pred) 
pred.shape

(931453, 24)

In [55]:
test_ids = np.array(pd.read_csv('../input/santander-pr/test.csv',usecols=['ncodpers'])['ncodpers'])

    
    

In [ ]:
products.get(657640)

In [57]:
final_preds = []
count = 10
for idx,predicted in enumerate(pred):
    ids = test_ids[idx]
    #print("ids", ids)
    top_product = targetcols[predicted]
    #print("top", top_product)
    used_product = recommendation_product.get(ids, [])
    #print("used", used_product)
    new_top_product = []
    for product in top_product:
        if product not in used_product:
            new_top_product.append(product)
        if len(new_top_product) == 5:
            break
    #print("new", new_top_product)
    final_preds.append(' '.join(new_top_product))
    

In [ ]:
submission = pd.DataFrame({'ncodpers':data_test1['ncodpers'],'changed':final_preds})


In [ ]:
submission.to_csv('submission2.csv', index=False)
#print(datetime.datetime.now()-start_time)

In [57]:
len(final_preds)

43

In [ ]:
final_preds[1]